In [ ]:
%reset -f

In [ ]:
import pandas as pd
import zipfile
import io
import requests
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve

In [ ]:
url = 'https://storage.googleapis.com/ds-publico/IA/loan_default.csv.zip'
response = requests.get(url)

with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
    with zip_file.open('loan_default.csv') as csv_file:
        df = pd.read_csv(csv_file)

In [ ]:
df.info()

In [ ]:
# Após uma análise dos dados (omitida nesse script) vamos manter apenas algumas das colunas fornecidas, apneas as de maior impacto, pela brevidade do trabalho.

keep = ['Status',
        'approv_in_adv',
        'credit_type',
        'loan_purpose',
        'age',
        'co_applicant_credit_type',
        'submission_of_application',
        'lump_sum_payment',
        'loan_amount',
        'income',
        'Credit_Score']

df_modelo = df[keep]
df_modelo.info()

In [ ]:
# Fazendo algumas limpezas finais nos dados

# Removendo todos os individuos com renda zero ou nula
df_modelo = df_modelo.query("income != 0").dropna(subset=['income'])

# Removendo todos os individuos em que loan_purpose é nulo
df_modelo = df_modelo.dropna(subset=['loan_purpose'])

# Removendo todos os individuos em que approv_in_adv é nulo
df_modelo = df_modelo.dropna(subset=['approv_in_adv'])

df_modelo.info()

In [ ]:
# Separando a variável dependente das explicativas
x = df_modelo.drop(columns=['Status'])
y = df_modelo['Status']

# Transformando as variáveis continuas (menos score) em logaritmo natural (linearização)
x['loan_amount'] = np.log(x['loan_amount'])
x['income'] = np.log(x['income'])

# Transformando as variaveis categoricas em dummies
categoricas = ['approv_in_adv',
               'credit_type',
               'loan_purpose',
               'age',
               'co_applicant_credit_type',
               'submission_of_application',
               'lump_sum_payment']

x = pd.get_dummies(x, columns=categoricas, drop_first=True)

In [ ]:
# Separando o df em treino e teste (80/20)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
logistic_model = LogisticRegression(random_state=42)
logistic_model.fit(x_train, y_train)
previsoes = logistic_model.predict(x_test)

In [ ]:
# Idenficando a probabilidade otima para corte entre as categorias (roc_curve)
# Ponto onde a taxa de falsos positivos é a mais baixa possível, enquanto a taxa de verdadeiros positivos é a mais alta possível

probs_train = logistic_model.predict_proba(x_train)
fpr, tpr, thresholds = roc_curve(y_train, probs_train[:, 1])
distancia = np.sqrt(fpr**2 + (1 - tpr)**2)
limiar_otimo = thresholds[np.argmin(distancia)]

print("Limiar ótimo de probabilidade:", limiar_otimo)

In [ ]:
probs_test = logistic_model.predict_proba(x_test)
previsoes = (probs_test[:, 1] >= limiar_otimo).astype(int)

In [ ]:
matriz_confusao = confusion_matrix(y_test, previsoes)

plt.figure(figsize=(4, 4))
sns.heatmap(matriz_confusao, annot=True, fmt='d', cmap='Blues', 
            xticklabels=logistic_model.classes_,
            yticklabels=logistic_model.classes_)
plt.xlabel('Previsto')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
f1_ponderado = f1_score(y_test, previsoes, average='weighted')
print("F1 Score ponderado:", f1_ponderado)

In [ ]:
# A probabilidade da classe positiva (status = 1) está na segunda coluna
probs_mau_pagador = probs_test[:, 1]

# Adicionando as probabilidades ao DataFrame X_train_dummies
df_resultado = pd.DataFrame({
    'X_test': x_test.index,  # Índice do DataFrame X_test_dummies
    'y_test': y_test,  # Valor real
    'Probabilidade_estimada': probs_mau_pagador,  # Probabilidade estimada
    'Valor_atribuido': previsoes  # Valor atribuído pelo modelo
})

In [ ]:
x_test.info()